# Pandathon Serasa Experean

Ana Ellen Deodato Pereira da Silva - 800206  
Gabrielly Castilho Guimarães - 805757   
João Pedro Trevisan - 769774  
Vinícius Gonçalves Perillo - 800219  

---

O desafio consiste em realizar o scraping de páginas de municipios brasileiros em busca de mapear os secretários de cada repartiçao. Estas pessoas sao expostas politicamente, portanto em busca de trasnparencia nas operacoes bancarias relacionadas a estes funcionarios publicos é necessário construir uma base de dados

## Importacoes e Instalacoes

Para esta tarefa faremos o uso do pacote beautifulsoup para o scraping, junto do pacore requests para tentar conexao com as paginas web. Alem disso expressoes regulares nos ajudaram nos scrapings e a biblioteca pandas nos permitira operar os dados como dataframes.

In [ ]:
#!pip install unidecode
#!pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import requests as requests
import re
import pandas as pd
from unidecode import unidecode


## Importacaoes de dados e processamentos

Neste ponto estamos recebendo o csv do IBGE que contem os nomes de todos os municipios do Brasil, ele é nosso ponto de partida para para encontrar os sites de cada municipio. criamos tambem uma lista de sinonimos para atacarmos as urls de maneira mais inteligente.

In [ ]:
df = pd.read_csv('data/UF_MINICIPIO.csv')

df

In [ ]:
df['nome_limpo'] = df['NOME_MUNICIPIO'].str.lower().str.replace(' ', '').str.replace("'",'').apply(unidecode)

df

In [ ]:
corpo = df['nome_limpo'] + pd.Series(['.']*df.shape[0]) + df['UF'].str.lower()
url = pd.Series(["http://www."]*df.shape[0]) + corpo + pd.Series([".gov.br"]*df.shape[0])

df['URL'] = url
df

In [ ]:
sinonimos = ['secretarias', 'orgaos', 'entidades', 'pagina', 'portal']

## Criacao das funcoes auxiliares

Agora sao criadas as funcoes para o trabalho de scraping.

A funcao valida_url() é usada para visitar uma possível url e checar se há retorno, isso é importante para atacarmos apenas sites válidos.

A funcao get_secretarias_url() toma a landing page de cada site e entao separa os links para a pagina que agrega a pagina de secretarias em cada site.

A funçao get_secretaria_home() busca a página que guarda o link para a página de secretarias e busca o link para cada secretaria individual.

O uso agregado destas funçoes nos permite encontrar urls válidas, encontrar onde está a página que agrega as secretarias e entao visitar cada uma delas.

In [ ]:
def valida_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except:
        return None





In [ ]:
def get_secretarias_url(text):
    if text != None:
        hrefs= []
        soup = BeautifulSoup(text, 'html.parser')

        #acha a secretaria de educacao
        links = soup.find_all('a', href=re.compile(r'educacao', flags=re.IGNORECASE))

        #print(links)

        #reconhece estruturas

        href_geral = ''
        class_geral = ''
        string_geral = ''
        title_geral = ''



        l = links[0]['href'].find('educacao')

        if  links[0]['href'][l-1] == '/':
            href_geral = links[0]['href'].replace('educacao', '')
        else:
            href_geral = links[0]['href'][:l-3]

        #print(href_geral)

        try:
            class_geral = re.compile(rf'{links[0]["class"].replace("educacao", "")}', flags=re.IGNORECASE)

        except:
            class_geral = None

        try:
            title_geral = re.compile(rf'{links[0]["title"].replace("educacao", "")}', flags=re.IGNORECASE)

        except:
            title_geral = None


        links = soup.find_all('a', href= re.compile(rf'{href_geral}', flags=re.IGNORECASE))

        #print(links)

        if links:
            for link in links:
                link = link["href"]
                hrefs.append(link if link.startswith('http') else url + link)

            return hrefs







In [ ]:
def get_secretarias_home(text):
    if text != None:
        soup = BeautifulSoup(text, 'html.parser')

        #caso secretarias em pagina especifica
        for s in sinonimos:
            links = soup.find_all('a', href=re.compile(rf'^(?!.*noticia).*{s}', flags=re.IGNORECASE))
            if links:
                for link in links:
                    link = link["href"]
                    if link.endswith(s) or link.endswith(s+'/'):
                        text = valida_url(link if link.startswith('http') else url + link)
                        return get_secretarias_url(text)


        #caso tem links de secretaria no homepage
        links = soup.find_all('a', href=re.compile(rf'^(?!.*noticia).*secretaria', flags=re.IGNORECASE))
        if links:
            return get_secretarias_url(text)




In [ ]:
def separa_nome_arquivo(link):
    l = link.find('educacao')
    return link[l:].replace('/', '-')

In [ ]:
def save_sec_page(file_name, link):
    fil = open('pages/' + file_name + '.txt', mode= 'w')
    fil.write(valida_url(url))
    fil.close()



Por fim, o loop principal de nossa aplicacao as funcoes preparam o nome do arquivo, puxa e salva o arquivo html em si e a funcao final process_url() agrega todas as outras, visitando as paginas corretas e armazenando cada uma delas.

In [ ]:
def process_url(url, municipio, uf):


   links = get_secretarias_home(valida_url(url))


   print(links)


   sep = pd.Series(['-']* len(link))
   ufs = pd.Series([uf]* len(link))
   municipios = pd.Series([municipio]* len(links))
   file_names = ufs + sep + municipios + sep + pd.Series(links)


   df = pd.DataFrame(dict(uf= ufs,
                     municipio= municipios,
                     file_name= file_names))


   for i in range(len(links)):
       try:
           save_sec_page(file_names[i], links[i])
       except Exception() as e:
           raise e


   df.to_csv('result.csv', mode='a', header= False)


In [ ]:
pd.DataFrame(dict(uf= [],
            municipio= [],
        	link= [])).to_csv('result.csv', index=False, mode='w')

In [ ]:
for i in df.index:
	line = df.loc[i, :]
	try:
    	process_url(line['URL'], line['NOME_MUNICIPIO'], line['UF'])
	except:
    	pass

Streaming output truncated to the last 5000 lines.
5566        http://www.tupirama.to.gov.brunidadesaude.php
5567      http://www.tupiratins.to.gov.brunidadesaude.php
5568    http://www.wanderlandia.to.gov.brunidadesaude.php
5569         http://www.xambioa.to.gov.brunidadesaude.php
Length: 5570, dtype: object, 0       http://www.acrelandia.ac.gov.brunidadeacaosoci...
1       http://www.assisbrasil.ac.gov.brunidadeacaosoc...
2       http://www.brasileia.ac.gov.brunidadeacaosocia...
3        http://www.bujari.ac.gov.brunidadeacaosocial.php
4       http://www.capixaba.ac.gov.brunidadeacaosocial...
                              ...                        
5565    http://www.tocantinopolis.to.gov.brunidadeacao...
5566    http://www.tupirama.to.gov.brunidadeacaosocial...
5567    http://www.tupiratins.to.gov.brunidadeacaosoci...
5568    http://www.wanderlandia.to.gov.brunidadeacaoso...
5569    http://www.xambioa.to.gov.brunidadeacaosocial.php
Length: 5570, dtype: object, 0       http://www.ac

Nosso objetivo por fim foi coletar a página html que guarda os nomes de cada secretário e nao apenas seus nomes.

Na impossibildade de realizar uma tarefa mais complexa, nosso foco foi realizar uma entrega escalável. Com os arquivos html, é possível realizar um processamento para extrair os nomes das pessoas.

Um algoritmo de PLN é capaz de fazer isso e pode ser otimizado para entregar resultados otimizados.

In [ ]:
import nltk

# Baixar o conjunto de dados necessário para o NLTK
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://maragogi.al.gov.br/secretarias/secretaria-de-educacao-seduc/'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Exemplo: Encontrar todos os links na página
    links = soup.find_all('a')
    for link in links:
        print(link.get('href'))  # Imprime os URLs dos links encontrados
else:
    print('Falha ao acessar a página.')

print(soup)

In [ ]:
texto = soup.text

tokens = nltk.word_tokenize(texto)
pos_tags = nltk.pos_tag(tokens)

chunks = nltk.ne_chunk(pos_tags)

# Procurar por entidades nomeadas do tipo 'PERSON'
for chunk in chunks:
    if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
        print( ' '.join(c[0] for c in chunk.leaves()))

Municipal
Educação
Maragogi
Portal Oficial
Gestão Municipal Ir
Acessibilidade Aumentar Fonte Diminuir Fonte Contraste Transparência Fiscal Início
Órgãos Conselhos Municipais Município Visite Maragogi História Feriados Municipais Links
Telefones Úteis Projeto Orla Sala
Empreendedor Legislação Lei Orgânica
Município Regulamentação
Serviços Secretaria Municipal
Educação Início
Órgãos Secretaria Municipal
Educação Secretaria Municipal
Educação
Arthur Cavalcante Bezerra Secretário Informações
Expediente Segunda
Educação
Adultos
V
Projeto Político Pedagógico
Plano
Ação
Criança
Educação Infantil
Educação Infantil
Adultos
Censo Escolar
Rede Municipal
Escola Cívico-Militar
Escola
Tayná Stephanie Sampaio Donato Informações
Centro
Expediente Segunda
Maragogi
Educação Notícias
Qualidade Educacional
Alagoas Notícias
Educação Infantil
Machado Filho
Maragogi
Horário
Rápido Glossário Mapa
Site Perguntas Frequentes Manual
Navegação Política
Sociais
Maragogi
Para
